<a href="https://colab.research.google.com/github/stefanoridolfi/ML_From_scratch/blob/master/ch_1_10_perceptron.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [91]:
# FROM ch_1_1.ipynb##########################
##############################################
import pandas as pd
from random import seed
from random import randrange
from csv import reader
from math import sqrt,exp

''' if no headers in csv
def load_csv(filename_url):
  CSV_url=filename_url
  pdfile=pd.read_csv(CSV_url,header=None)
  dataset=pdfile.values.tolist()
  return dataset
  '''
'''if header in csv
#CVS with headers ##################
def load_csv(filename_url):
  CSV_url=filename_url
  pdfile=pd.read_csv(CSV_url,sep=';',nrows=48)
  #pdfile=pd.read_csv(CSV_url,sep=';')
  headers=pdfile.head()
  dataset=pdfile.values.tolist()
  return dataset, list(headers)
  '''
def load_csv(filename_url):
  CSV_url=filename_url
  pdfile=pd.read_csv(CSV_url,header=None)
  dataset=pdfile.values.tolist()
  return dataset


##################

# Dataset are in https://github.com/jbrownlee/Datasets

#dataset Sonar
CSV_url='https://raw.githubusercontent.com/jbrownlee/Datasets/master/sonar.csv'

dataset= load_csv(CSV_url)
print('Loaded data file {0} with {1} rows and {2} columns\n\n'.format(CSV_url, len(dataset),len(dataset[0])))

# Convert string column to float
def str_column_to_float(dataset, column):
  for row in dataset:
    row[column] = float(row[column].strip())

# Convert string to int (this is because we need class name that is number, like last column in this dataset)
def str2int(dataset,colonna):
  dataset_binario=list(dataset)
  classes=[]
  for row in dataset:
    classes.append(row[colonna])
  unique=set(classes)
  lookup=dict()
  for i, values in enumerate(unique):
    lookup[values]=i
  for row in dataset_binario:
    row[colonna]=lookup[row[colonna]]
  return dataset_binario
  



# function that splits dataset in two parts, train and test
def train_test_split(dataset,split=0.7):
  train_list=[]
  train_size=int(split*len(dataset))
  test_list=list(dataset)
  while len(train_list)  <train_size:
    index=randrange(len(test_list))
    #print("index=",index)
    elem=test_list.pop(index)
    train_list.append(elem)
  return train_list, test_list

# Function that splits  dataset in k folds : result is a list of list of folds, inside is fold there are lists that are the columns of dataset
def cross_validation_split(dataset, folds):
  dataset_split=list()
  dataset_copy=list(dataset)
  print("len dataset", len(dataset))
  fold_size=int(len(dataset)/folds)
  for i in range(folds):
    fold=[]
    while len(fold)<fold_size:
      index=randrange(len(dataset_copy))
      elem=dataset_copy.pop(index)
      fold.append(elem)
    dataset_split.append(fold)
  return dataset_split

  # function that calculates accuracy
def accuracy_m(actual, predicted):
  correct=0
  for i in range(len(actual)):
    if actual[i]==predicted[i]:
      correct+=1
    accuracy=(correct/float(len(actual)))*100
  return accuracy

# Make a prediction for logistic regression: function that calculates the output Y using coeffcients b0, b1 ...etc
def predict(row, weigths):
  activation=weigths[0]
  for i in range(len(row)-1):
    activation+= weigths[i + 1] * row[i]
  if activation >0:
    result=1
  else:
    result=0
  return result

# Estimate weigts using stochastic stochastic gradient descent
def calc_weigths(train, lr, epoches):
  weigths = [0.0 for i in range(len(train[0]))]
  for epoch in range(epoches):
    for row in train:
      prediction=predict(row,weigths)
      error=row[-1]-prediction
      weigths[0]=weigths[0]+lr*float(error)
      for i in range(len(row)-1):
        weigths[i+1]=weigths[i+1]+lr*error*row[i]
  return weigths







###############START code###########################
seed(1)
n_folds=3
lr=0.01
epoches=200
dataset_binario=str2int(dataset,len(dataset[0])-1)
dataset_split_folds=cross_validation_split(dataset_binario,n_folds)
train, test =train_test_split(dataset_binario)


################# Case dataset splitted in 2 parts########################################
weigths = calc_weigths(train, lr, epoches)
predicted=[]
actual=[int(row[-1]) for row in test]
for row in test:
  prediction=predict(row,weigths)
  predicted.append(prediction)
#print("len actual",len(actual),"le predicted",len(predicted))
accuracy=accuracy_m(actual, predicted)
print("accuracy split in 2 parts case: %.2f%%" % accuracy)

################# Case dataset splitted in folds########################################
scores=[]
for fold in dataset_split_folds:
  train_set=list(dataset_split_folds)
  train_set.remove(fold)
  train_set=sum(train_set,[])
  test_set=fold
  weigths = calc_weigths(train_set, lr, epoches)
  predicted=[]
  actual=[row[-1] for row in test_set]
  for row in test_set:
    prediction=predict(row,weigths)
    predicted.append(prediction)
  accuracy=accuracy_m(actual, predicted)
  scores.append(accuracy)
accuracy_mean=sum(scores)/float(len(scores))
print("")
print("accuracies:",scores,"\nmean accuracy split in", n_folds, "parts case: %.2f%%" % accuracy_mean)






Loaded data file https://raw.githubusercontent.com/jbrownlee/Datasets/master/sonar.csv with 208 rows and 61 columns


len dataset 208
accuracy split in 2 parts case: 71.43%

accuracies: [79.71014492753623, 72.46376811594203, 71.01449275362319] 
mean accuracy split in 3 parts case: 74.40%
